In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import locale
locale.setlocale(locale.LC_ALL,'')

'Spanish_Mexico.1252'

In [2]:
columnas = pd.read_excel("data.xlsx",'data', nrows=1).columns
i = 0
for n in columnas:
    print(f'Col {i}:',n)
    i+=1

Col 0: Número de movimiento 
Col 1: Documento 
Col 2: Clave de artículo 
Col 3: Descripción 
Col 4: Línea 
Col 5: Clave cliente/proveedor 
Col 6: Concepto 
Col 7: Fecha 
Col 8: Vendedor 
Col 9: Precio 
Col 10: Costo 
Col 11: Almacén 
Col 12: Tipo de documento 
Col 13: Costo operado 
Col 14: Cantidad 


In [2]:
df = pd.read_excel("data.xlsx",'data')
df["Fecha "] = pd.to_datetime(df["Fecha "], dayfirst=True)

In [3]:
dfA1 = df[df["Almacén "] == 1][["Almacén ", "Fecha ","Cantidad "]]
dfA2 = df[df["Almacén "] == 2][["Almacén ", "Fecha ","Cantidad "]]